RooFit Tutorial: http://roofit.sourceforge.net/docs/tutorial/intro/roofit_tutorial_intro.pdf

In [1]:
import ROOT as R
from iminuit import Minuit
from iminuit.cost import ExtendedBinnedNLL
import numpy as np
import numba as nb
from matplotlib import pyplot as plt
from progressbar import progressbar
from pyik.mplext import plot_hist
from IPython.display import display
from scipy.stats import norm

Welcome to JupyROOT 6.23/01


In [2]:
# numba currently does not support scipy, so we cannot access
# scipy.stats.norm.cdf in a JIT'ed function. As a workaround,
# we wrap special functions from scipy to implement the needed
# functions here.
def get(name, narg):
    from numba.extending import get_cython_function_address
    import ctypes
    addr = get_cython_function_address("scipy.special.cython_special", name)
    functype = ctypes.CFUNCTYPE(ctypes.c_double, *([ctypes.c_double] * narg))
    return functype(addr)


erf = get("__pyx_fuse_1erf", 1)


@nb.njit
def norm_cdf(x, mu, sigma):
    z = (x - mu) / (sigma * np.sqrt(2))
    r = np.empty_like(x)
    for i, zi in enumerate(z):
        r[i] = 0.5 * (1.0 + erf(zi))
    return r


def model(x, sig, bkg):
    return sig * norm_cdf(x, 0.5, 0.1) + bkg * x


xe = np.linspace(0, 1, 21)
n = np.diff(model(xe, 20, 200))
vbkg = np.diff(model(xe, 0, 200))

In [3]:
rng = np.random.default_rng(1)

h = R.TH1D("h", "", 20, 0, 1)

x = R.RooRealVar("x", "", 0, 1)
mu = R.RooRealVar("mu", "", 0.5)
sigma = R.RooRealVar("sigma", "", 0.1)
sig = R.RooRealVar("sig", "", 0, 0, 1000)
gauss = R.RooGaussian("gauss", "", x, mu, sigma)
bkg = R.RooRealVar("bkg", "", 20, 0, 1000)
flat = R.RooUniform("uni", "", x)
model = R.RooAddPdf("model", "", R.RooArgList(gauss, flat), R.RooArgList(sig, bkg))
hist = R.RooDataHist("hist", "", x, R.RooFit.Import(h))

values = []
for imc in range(10):
    n = rng.poisson(vbkg)
    for i, ni in enumerate(rng.poisson(vbkg)):
        hist.set(1 + i, ni)
    hist.Print("v")
    model.fitTo(hist)
    values.append(sig.getVal())


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt

DataStore hist ()
  Contains 20 entries
  Observables: 
    1)  x = 0.975  L(0 - 1) B(20)  ""
Binned Dataset hist ()
  Contains 20 bins with a total weight of 20
  Observables:     1)  x = 0.975  L(0 - 1) B(20)  ""
[#1] INFO:Minization -- p.d.f. provides expected number of events, including extended term in likelihood.
[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:Minization --  The following expressions have been identified as constant and will be precalculated and cached: (gauss,uni)
 **********
 **    1 **SET PRINT           1
 **********
 **********
 **    2 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 bkg          2.00000e

In [4]:
print(values)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
